# Исследование надежности заемщиков.

Для эффективной работы банку нужно оценить способность потенциального заёмщика вернуть кредит. На статистике о платёжеспособности клиентов мы посмотрим, от чего зависит способность клиента вернуть кредит в срок.

**Цель исследования** — проверить четыре гипотезы:
1. Вероятность возврата кредита в срок зависит от количества детей клиента.
2. Вероятность возврата кредита в срок зависит от семейного положения клиента. 
3. Вероятность возврата кредита в срок зависит от уровня дохода клиента.
4. Цели кредита влияют на вероятность возврат кредита в срок.

**Ход исследования**

Исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.

###  Обзор данных

Составим первое представление о данных о платежеспособности клиентов.

In [1]:
# импорт библиотек
import pandas as pd
import numpy as np


In [2]:
# чтение файла с данными и сохранение в df 
df = pd.read_csv('data (2).csv')
# получение первых 10 строк таблицы df
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Выводы**

В каждой строке таблицы — данные о клиенте, его кредите и задолженности по возврату.  Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

### Заполнение пропусков

Посчитаем, сколько в таблице пропущенных значений.

In [4]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Мы обнаружили пропуски в `days_employed ` и `total_income`. Проверим, какую долю составляют пропущенные значения в каждом из столбцовс пропусками.


In [5]:
df.isna().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

Пропущенные значения в каждом из столбцов с пропусками составляют около 10%. В `days_employed ` и `total_income` мы имеем дело с количесвтенными переменными. Такие пропуски заполняют характерными
значениями. Это значения, характеризующие состояние выборки. 

Чтобы примерно оценить типичные значения выборки, годятся среднее арифметическое или медиана.Чтобы избежать влияния выбросов, будем использовать медаину.




В столбце `total_income` заполним пропуски  медианой в зависимости от типа занятости

In [6]:
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform("median"))


Найдем медианное значение для столбца `days_employed`.

In [7]:
df['days_employed'].sort_values().median()

-1203.369528770489

Мы столкнулись с аномалией - отрицательное количество дней трудового стажа в столбце days_employed. Для реальных данных это нормально. Обработаем значения в столбцах с аномалиями.

### Проверка данных на аномалии и исправления.

Возможно, при записи столбца `days_employed` произошлел технический сбой. Посмотим, какие значения встрчаются в столбце


In [8]:
df['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

Большая часть значаений - отрицательные. Также приссутсвуют выбросы. Возьмем абсолютные значения и найдем медиану. 

In [9]:
 df['days_employed'] = abs(df['days_employed'])

In [10]:
days_employed_median = df['days_employed'].sort_values().median()

Заполним пропуски медианой 

In [11]:
df['days_employed'] = df['days_employed'].fillna(days_employed_median)

In [12]:
df['days_employed'].value_counts()


2194.220567      2175
329951.594147       1
886.253127          1
2539.534295         1
390574.985524       1
                 ... 
1320.841444         1
1394.302246         1
2325.720832         1
4086.407828         1
1636.419775         1
Name: days_employed, Length: 19351, dtype: int64

Проверим значения в критических для анализа столбцах с категориальными данными - дети, семейное положение и цели. Сначала найдем уникальные значения столбца `children`.

In [13]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце `children` два странных значения - 20 и -1. Скорее всего, оба значения стали результатом ошибки. Поскольку на значения '20' и '-1' в столбце `children` приходится меньше 1% данных, их удаление не повлияет на финальный результат. 


In [14]:
df = df.loc[df['children'] != 20]
df = df.loc[df['children'] != -1]


Проверим удаление.

In [15]:
df['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Теперь найдем уникальные значения столбца о семейном положении.

In [16]:
df['family_status'].value_counts()

женат / замужем          12302
гражданский брак          4160
Не женат / не замужем     2799
в разводе                 1189
вдовец / вдова             952
Name: family_status, dtype: int64

Со значениями все впорядке.

Посмотрим, что встречается в целях.

In [17]:
df['purpose'].value_counts()

свадьба                                   796
на проведение свадьбы                     772
сыграть свадьбу                           769
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   651
операции с жильем                         648
операции с коммерческой недвижимостью     646
жилье                                     642
покупка жилья                             641
покупка жилья для семьи                   640
недвижимость                              632
строительство собственной недвижимости    628
операции со своей недвижимостью           626
строительство жилой недвижимости          622
строительство недвижимости                620
покупка своего жилья                      619
покупка недвижимости                      619
ремонт жилью                              609
покупка жилой недвижимости                603
на покупку своего автомобиля              504
заняться высшим образованием      

Кажется, все ок, но встречаются неявные дубликаты вроде "автомобили","сделка с автомобилем","приобретение автомобиля" и тд. Заменим это в следующем шаге.

###  Изменение типов данных.

Также заменим вещественный тип данных в столбце total_income на целочисленный.


In [18]:
df['total_income']=df['total_income'].astype('int')


### Удаление дубликатов.

Найдем строки-дубликаты.

In [19]:
df.duplicated().sum()

54

Удалим строки-дубликаты

In [20]:
df = df.drop_duplicates().reset_index(drop=True)

Обработаем неявные дубликаты.Например, в столбце education есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к одному регистру.

In [21]:
df['education']= df['education'].str.lower()


Проверим

In [22]:
df['education'].value_counts()

среднее                15091
высшее                  5228
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

Еще раз посмотрим 
уникальные значения столбца 'purpose'


In [23]:
df['purpose'].value_counts()

свадьба                                   792
на проведение свадьбы                     768
сыграть свадьбу                           764
операции с недвижимостью                  672
покупка коммерческой недвижимости         659
покупка жилья для сдачи                   650
операции с жильем                         647
операции с коммерческой недвижимостью     645
жилье                                     641
покупка жилья                             640
покупка жилья для семьи                   637
недвижимость                              631
строительство собственной недвижимости    628
операции со своей недвижимостью           623
строительство жилой недвижимости          621
строительство недвижимости                619
покупка своего жилья                      619
покупка недвижимости                      616
ремонт жилью                              604
покупка жилой недвижимости                602
на покупку своего автомобиля              504
заняться высшим образованием      

Напишем функцию для замены дупликатов.

In [24]:
def replace_wrong_purpose(wrong_purposes, correct_purpose):
    for wrong_purpose in wrong_purposes:
        df['purpose'] = df['purpose'].replace(wrong_purpose,correct_purpose)
 

Объединим кредиты, связанные со свадьбой

In [25]:
duplicates = ['на проведение свадьбы', 'сыграть свадьбу']
correct_purpose = 'свадьба'
replace_wrong_purpose(duplicates,correct_purpose)

Объединим кредиты, связанные с недвижимостью

In [26]:
duplicates = ['операции с недвижимостью','покупка коммерческой недвижимости','покупка жилья для сдачи','операции с жильем','операции с коммерческой недвижимостью','покупка жилья','жилье','покупка жилья для семьи','строительство собственной недвижимости','недвижимость','операции со своей недвижимостью','строительство жилой недвижимости','покупка недвижимости','покупка своего жилья','строительство недвижимости','ремонт жилью','покупка жилой недвижимости']
correct_purpose = 'недвижимость'
replace_wrong_purpose(duplicates,correct_purpose)

Объединим кредиты, связанные с автомобилем:

In [27]:
duplicates = ['на покупку своего автомобиля', 'сделка с подержанным автомобилем','свой автомобиль','автомобили','на покупку подержанного автомобиля','на покупку автомобиля','приобретение автомобиля','сделка с автомобилем']#correct_purpose = 'автомобиль' 
correct_purpose = 'автомобиль'
replace_wrong_purpose(duplicates,correct_purpose)

Объединим кредиты, связанные с образованием:

In [28]:
duplicates = ['заняться высшим образованием','дополнительное образование','высшее образование','получение образования','заняться образованием','профильное образование','получение высшего образования','получение дополнительного образования','образование ']
correct_purpose = 'образование' 
replace_wrong_purpose(duplicates,correct_purpose)

Проверим, какие значения остались

In [29]:
df['purpose'].value_counts()

недвижимость    10754
автомобиль       4281
образование      3989
свадьба          2324
Name: purpose, dtype: int64

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим новый датафрейм, в котором  каждому уникальному значению из education соответствует уникальное значение education_id

In [30]:
education_dict = df[['education','education_id']]


Создадим новый датафрейм, в котором каждому уникальному значению из family_status соответствует уникальное значение family_status_id 

In [31]:
family_dict = df[['family_status','family_status_id']]


Удалим из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id. 

In [32]:
df = df[['children','days_employed','dob_years','education_id', 'family_status_id','gender','income_type','debt','total_income','purpose']]

In [33]:
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,недвижимость
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,автомобиль
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,недвижимость
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,свадьба
5,0,926.185831,27,0,1,M,компаньон,0,255763,недвижимость
6,0,2879.202052,43,0,0,F,компаньон,0,240525,недвижимость
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,свадьба
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,недвижимость


### Категоризация дохода.

Создадим столбец total_income_category с категориями:
0–30000 — 'E';
30001–50000 — 'D';
50001–200000 — 'C';
200001–1000000 — 'B';
1000001 и выше — 'A'.

In [34]:
def total_income_category(total_income):
    if total_income <= 30000:
        return 'E'
    if 30001 <= total_income <= 50000:
        return 'D'
    if 50001 <= total_income <= 200000:
        return 'C'
    if 200001 <= total_income <= 1000000:
        return 'B'
    return 'A' 

In [35]:
df['total_income_category'] = df['total_income'].apply(total_income_category)
df.head(30)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,недвижимость,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,автомобиль,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,недвижимость,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,свадьба,C
5,0,926.185831,27,0,1,M,компаньон,0,255763,недвижимость,B
6,0,2879.202052,43,0,0,F,компаньон,0,240525,недвижимость,B
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование,C
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,свадьба,C
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,недвижимость,C


### Обстоятельство и задолженность.

Проверим, как семейное положение влиияет на задолженность.


In [36]:
pvt_family = pd.pivot_table(df, index = 'family_status_id', columns = 'debt', values = 'dob_years', aggfunc ='count')
pvt_family['share'] =(pvt_family[1]/ (pvt_family[0]+pvt_family[1])*100).round(2)
pvt_family 

debt,0,1,share
family_status_id,,,
0,11339,927,7.56
1,3761,385,9.29
2,888,63,6.62
3,1105,84,7.06
4,2523,273,9.76


Чаще остальных с выплатой кредитов проблемы у двух категорий - гражданский брак и не женат/не замужем. Лучше всех с выплатой кредитов у категории вдовец/вдова и женат/замужем.

Проверим, как дети влиияют на задолженность.

In [37]:
pvt_children = pd.pivot_table(df,index ='' 'children',columns = 'debt', values ='dob_years',aggfunc ='count')
pvt_children['share'] =(pvt_children[1]/ (pvt_children[0]+pvt_children[1])*100).round(2)
pvt_children 

debt,0,1,share
children,,,
0,13044.0,1063.0,7.54
1,4365.0,444.0,9.23
2,1858.0,194.0,9.45
3,303.0,27.0,8.18
4,37.0,4.0,9.76
5,9.0,NaN,NaN


Лучше всего возвращают кредиты заемщики без детей

Прверим вероятность возврата кредита в срок в зависимости от уровня дохода клиента.


In [38]:
pvt_income = pd.pivot_table(df,index = 'total_income_category',columns = 'debt', values ='dob_years',aggfunc ='count')
pvt_income['share'] =(pvt_income[1]/ (pvt_income[0]+pvt_income[1])*100).round(2)
pvt_income

debt,0,1,share
total_income_category,,,
A,23,2,8.00
B,4660,354,7.06
C,14585,1353,8.49
D,328,21,6.02
E,20,2,9.09


Хуже всего кредиты выпалчивают заемщики, доход которых ниже 30 тыс в месяц. Лучше всего - клиенты, чей доход превышает составляи от 30 до 50 тыс ежемесячно. 

Как влияет цель:

In [39]:
pvt_purpose = pd.pivot_table(df,index = 'purpose',columns = 'debt', values ='dob_years',aggfunc ='count')
pvt_purpose['share'] =(pvt_purpose[1]/ (pvt_purpose[0]+pvt_purpose[1])*100).round(2)
pvt_purpose

debt,0,1,share
purpose,,,
автомобиль,3881,400,9.34
недвижимость,9974,780,7.25
образование,3620,369,9.25
свадьба,2141,183,7.87


Кто берет кредит на свадьбу или на недвижимость чаще возвращают кредит, чем те, кто берет его на автомобиль или образование.

### Общий вывод.



***Семейное положение***: чаще остальных с выплатой кредитов проблемы у двух категорий - гражданский брак и не женат/не замужем. Лучше всех с выплатой кредитов у категории вдовец/вдова и женат/замужем.


***Доход***:уже всего кредиты выпалчивают заемщики, доход которых ниже 30 тыс в месяц. Лучше всего - клиенты, чей доход превышает составляи от 30 до 50 тыс ежемесячно. 

***Дети***:лучше всего возвращают кредиты заемщики без детей.

***Цель***:кто берет кредит на свадьбу или на недвижимость чаще возвращают кредит, чем те, кто берет его на автомобиль или образование.